In [18]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transfrorms
import torch.utils.data as data
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# 超参数
num_classes = 10
num_epoch = 5
batch_size = 100
learning_rate = 1e-4

# 数据加载
# dataset
train_data = torchvision.datasets.CIFAR10(
    root='pytorch-Learning/data/CIFAR-10',
    train=True,
    transform=transfrorms.ToTensor(),
    download=True
)
test_data = torchvision.datasets.CIFAR10(
    root='pytorch-Learning/data/CIFAR-10',
    train=False,
    transform=transfrorms.ToTensor()
)
# dataloader
train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=batch_size,  # 批大小，用于每次训练的样本数量
    shuffle=True  # 每步迭代时，要打乱dataset的顺序来取出batch_size个样本
)
test_loader = data.DataLoader(
    dataset=test_data,
    batch_size=batch_size,  # 批大小，用于每次训练的样本数量
    shuffle=False  # 每步迭代时，不要打乱dataset的顺序来取出batch_size个样本
)

Files already downloaded and verified


In [4]:
# 定义3x3卷积层
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels=in_channels, out_channels=out_channels, stride=stride, 
                     padding=1, bias=False, kernel_size=3)

# 残差块
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        self.con1 = conv3x3(in_channels, out_channels)
        self.bn1 = nn.BatchNorm2d(out_channels)
        # inplace=True 表直接覆盖原变量，可减少内存消耗，不影响结果
        self.relu = nn.ReLU(inplace=True) 
        self.con2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.con1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.con2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
